# Spark DataFrames & SQL - Interactive Learning Notebook

Welcome to Lesson 2 of the Spark Bootcamp! This notebook provides hands-on experience with Spark DataFrames and SQL operations.

## Learning Objectives

By the end of this notebook, you will:
- Understand DataFrame fundamentals and advantages over RDDs
- Create DataFrames from various sources
- Perform data transformations and aggregations
- Use Spark SQL for complex queries
- Apply window functions and analytics
- Optimize performance with caching and query plans

## Section 1: Setting Up Spark DataFrame Environment

In [ ]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Create Spark session
spark = SparkSession.builder \
    .appName("DataFrames-Learning") \
    .master("local[*]") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

print(f"Spark version: {spark.version}")
print(f"Spark context available: {spark.sparkContext is not None}")

## Section 2: DataFrame Creation Fundamentals

Let's start by creating DataFrames using different methods.

In [ ]:
# Method 1: Create DataFrame from Python data structures
employee_data = [
    ("Alice Johnson", 28, "Engineering", 85000),
    ("Bob Chen", 34, "Marketing", 65000),
    ("Charlie Davis", 29, "Engineering", 78000),
    ("Diana Rodriguez", 31, "Sales", 72000),
    ("Eve Wilson", 26, "Engineering", 92000)
]

columns = ["name", "age", "department", "salary"]
employees_df = spark.createDataFrame(employee_data, columns)

print("DataFrame created from list of tuples:")
employees_df.show()
employees_df.printSchema()

In [ ]:
# Method 2: Create DataFrame with explicit schema
schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("department", StringType(), True),
    StructField("salary", DoubleType(), True)
])

employees_typed_df = spark.createDataFrame(employee_data, schema)

print("DataFrame with explicit schema:")
employees_typed_df.printSchema()
print(f"Row count: {employees_typed_df.count()}")

In [ ]:
# Method 3: Create DataFrame from dictionaries
product_dicts = [
    {"id": "PROD001", "name": "Laptop", "category": "Electronics", "price": 999.99, "stock": 50},
    {"id": "PROD002", "name": "Mouse", "category": "Electronics", "price": 29.99, "stock": 200},
    {"id": "PROD003", "name": "Chair", "category": "Furniture", "price": 199.99, "stock": 30}
]

products_df = spark.createDataFrame(product_dicts)

print("DataFrame from dictionaries:")
products_df.show()
print(f"Columns: {products_df.columns}")

## Section 3: Basic DataFrame Operations

Learn the fundamental operations for working with DataFrames.

In [ ]:
# Basic operations
print("=== Basic DataFrame Operations ===")

# Select specific columns
print("\n1. Select name and salary:")
employees_df.select("name", "salary").show()

# Filter data
print("\n2. Filter Engineering employees:")
engineers = employees_df.filter(col("department") == "Engineering")
engineers.show()

# Sort data
print("\n3. Sort by salary descending:")
employees_df.orderBy(desc("salary")).show()

# Add calculated columns
print("\n4. Add annual bonus column (10% of salary):")
with_bonus = employees_df.withColumn("annual_bonus", col("salary") * 0.1)
with_bonus.select("name", "salary", "annual_bonus").show()

In [ ]:
# Aggregations
print("=== Aggregation Operations ===")

# Basic aggregations
print("\n1. Overall statistics:")
stats = employees_df.agg(
    count("*").alias("total_employees"),
    avg("salary").alias("avg_salary"),
    max("salary").alias("max_salary"),
    min("salary").alias("min_salary")
)
stats.show()

# Group by aggregations
print("\n2. Statistics by department:")
dept_stats = employees_df.groupBy("department").agg(
    count("*").alias("employee_count"),
    round(avg("salary"), 2).alias("avg_salary"),
    sum("salary").alias("total_payroll")
)
dept_stats.show()

## Section 4: Column Functions and Transformations

Explore the rich set of column functions available in Spark.

In [ ]:
# String functions
print("=== String Functions ===")

string_demo_df = employees_df.select(
    col("name"),
    upper(col("name")).alias("name_upper"),
    length(col("name")).alias("name_length"),
    split(col("name"), " ").getItem(0).alias("first_name"),
    split(col("name"), " ").getItem(1).alias("last_name")
)

string_demo_df.show(truncate=False)

In [ ]:
# Conditional logic with when/otherwise
print("=== Conditional Logic ===")

conditional_df = employees_df.select(
    col("name"),
    col("age"),
    col("salary"),
    when(col("age") < 30, "Young")
        .when(col("age") < 40, "Mid-career")
        .otherwise("Experienced").alias("age_group"),
    when(col("salary") > 80000, "High")
        .when(col("salary") > 70000, "Medium")
        .otherwise("Standard").alias("salary_band")
)

conditional_df.show()

## Section 5: Spark SQL Integration

Use SQL syntax for data analysis with temporary views.

In [ ]:
# Create temporary views for SQL queries
employees_df.createOrReplaceTempView("employees")
products_df.createOrReplaceTempView("products")

print("=== Spark SQL Queries ===")

# Basic SQL query
print("\n1. Basic SELECT with WHERE:")
result1 = spark.sql("""
    SELECT name, department, salary
    FROM employees
    WHERE salary > 75000
    ORDER BY salary DESC
""")
result1.show()

# Aggregation with GROUP BY
print("\n2. Aggregation with GROUP BY:")
result2 = spark.sql("""
    SELECT 
        department,
        COUNT(*) as employee_count,
        ROUND(AVG(salary), 2) as avg_salary,
        MAX(salary) as max_salary
    FROM employees
    GROUP BY department
    ORDER BY avg_salary DESC
""")
result2.show()

## Section 6: Window Functions and Advanced Analytics

Perform sophisticated analytics using window functions.

In [ ]:
# Window functions for ranking and analytics
print("=== Advanced Window Functions ===")

# Define windows
dept_window = Window.partitionBy("department").orderBy(desc("salary"))
overall_window = Window.orderBy(desc("salary"))

# Apply window functions
windowed_analysis = employees_df.select(
    col("name"),
    col("department"),
    col("salary"),
    
    # Ranking functions
    row_number().over(dept_window).alias("dept_rank"),
    rank().over(overall_window).alias("overall_rank"),
    dense_rank().over(dept_window).alias("dept_dense_rank"),
    
    # Analytical functions
    round(avg("salary").over(Window.partitionBy("department")), 2).alias("dept_avg"),
    lag("salary", 1).over(dept_window).alias("next_lower_salary")
)

windowed_analysis.orderBy("department", "dept_rank").show()

## Section 7: Performance Optimization

Learn about caching and query optimization.

In [ ]:
# Caching for performance
print("=== Caching and Performance ===")

# Cache DataFrame for reuse
employees_df.cache()
employees_df.count()  # Force caching

print("DataFrame cached for better performance in repeated operations")

# Query execution plan
print("\n=== Query Execution Plan ===")
complex_query = employees_df.filter(col("salary") > 75000).groupBy("department").avg("salary")
print("\nExecution plan for complex query:")
complex_query.explain(True)

## Section 8: Working with Complex Data Types

Handle arrays, maps, and nested structures.

In [ ]:
# Working with arrays and complex types
print("=== Complex Data Types ===")

# Create data with arrays
complex_data = [
    ("Alice", ["Python", "SQL", "Spark"]),
    ("Bob", ["Java", "Scala"]),
    ("Charlie", ["Python", "R", "SQL", "Tableau"])
]

complex_df = spark.createDataFrame(complex_data, ["name", "skills"])

print("\nOriginal complex data:")
complex_df.show(truncate=False)

# Work with arrays
print("\nArray operations:")
array_ops = complex_df.select(
    col("name"),
    col("skills"),
    size(col("skills")).alias("skill_count"),
    array_contains(col("skills"), "Python").alias("knows_python"),
    col("skills")[0].alias("primary_skill")
)
array_ops.show()

## Section 9: Integration with Pandas

Convert between Spark DataFrames and Pandas for visualization.

In [ ]:
# Convert to Pandas for visualization
print("=== Spark to Pandas Integration ===")

# Get department statistics
dept_stats_spark = employees_df.groupBy("department").agg(
    count("*").alias("employee_count"),
    round(avg("salary"), 0).alias("avg_salary")
)

# Convert to Pandas
dept_stats_pandas = dept_stats_spark.toPandas()

print("\nDepartment statistics (Pandas):")
print(dept_stats_pandas)

# Create visualizations
plt.figure(figsize=(12, 5))

# Employee count by department
plt.subplot(1, 2, 1)
plt.bar(dept_stats_pandas['department'], dept_stats_pandas['employee_count'])
plt.title('Employee Count by Department')
plt.xlabel('Department')
plt.ylabel('Employee Count')
plt.xticks(rotation=45)

# Average salary by department
plt.subplot(1, 2, 2)
plt.bar(dept_stats_pandas['department'], dept_stats_pandas['avg_salary'])
plt.title('Average Salary by Department')
plt.xlabel('Department')
plt.ylabel('Average Salary ($)')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## Summary and Next Steps

Congratulations! You've completed the Spark DataFrames & SQL interactive tutorial. You've learned:

### Key Concepts Covered:
1. **DataFrame Creation** - From various data sources and with different schemas
2. **Basic Operations** - Select, filter, sort, and aggregate data
3. **Column Functions** - String, numeric, date, and conditional operations
4. **Spark SQL** - Using SQL syntax for complex queries
5. **Window Functions** - Advanced analytics and ranking
6. **Performance Optimization** - Caching and execution plan analysis
7. **Complex Data Types** - Arrays, maps, and nested structures
8. **Integration** - Working with Pandas and visualization

### Next Steps:
1. **Practice with Exercises** - Complete the 7 hands-on exercises
2. **Explore Solutions** - Study the provided solution files
3. **Run Tests** - Validate your understanding with the test suite
4. **Advanced Topics** - Move on to streaming, MLlib, or advanced optimizations

### Resources:
- Exercise files in the `exercises/` directory
- Solution files in the `solutions/` directory
- Test your code with `make test`
- Use `make help` to see all available commands

Happy Spark learning! 🚀

In [ ]:
# Clean up
spark.stop()
print("Spark session stopped. Notebook complete!")